In [23]:
from fredapi import Fred
import pandas as pd
from datetime import datetime

# Setup
fred = Fred(api_key="YOUR API KEY HERE")

start_date = "1997-01-01"
# Get today's date and format it
end_date = datetime.today().strftime("%Y-%m-%d")
freq = "MS"

# Mapping: FRED series code → layman__ variable
fred_layman_dict = {
    "WTISPLC": "layman__oil_price",
    "GASREGCOVW": "layman__gas_price",
    "MORTGAGE30US": "layman__mortgage_rate_30yr",
    "MSPUS": "layman__median_home_price",
    "CUSR0000SAF1": "layman__cpi_grocery_prices",
    "RSAFS": "layman__total_retail_sales",
    "CUSR0000SEHF01": "layman__cpi_electricity",
    "CUSR0000SETA02": "layman__cpi_used_cars",
    "TOTALSL": "layman__total_consumer_credit",
    "TERMCBAUTO48NS": "layman__avg_auto_loan_rate_48mo",
    "SLOAS": "layman__total_student_loans",
    "RRVRUSQ156N": "layman__rental_vacancy_rate",
    "CUUR0000SETA03": "layman__cpi_fast_food",
    "CUSR0000SEEA": "layman__cpi_event_admission",
    "CUSR0000SETG01": "layman__cpi_airfare",
    "CUSR0000SEEB03": "layman__cpi_childcare",
    "CUSR0000SETG": "layman__cpi_transit_fares",
    "UMCSENT": "layman__consumer_sentiment_index",
    "PSAVERT": "layman__personal_savings_rate",
    "UNRATE": "layman__unemployment_rate",
    "FEDFUNDS": "layman__federal_funds_rate",
    "LNS14000036": "layman__youth_unemployment_rate",
}

fallback_fred_layman_dict = {
    # ... keep all the existing successful entries
    "CPIUFDSL": "layman__food_price_index",               # replaces grocery
    "CPIUFDINE": "layman__food_away_from_home_index",     # replaces fast food
    "CPIEDUCNS": "layman__education_and_childcare_cpi",   # replaces childcare
    "CPIAUCSL": "layman__general_cpi_index",   # General CPI (replacement for grocery)
    "CPIMEDSL": "layman__medical_services_index",  # Medical costs as household pressure
}

# Download series
date_index = pd.date_range(start=start_date, end=end_date, freq=freq)
df = pd.DataFrame(index=date_index)

for code, alias in fred_layman_dict.items():
    try:
        series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
        df[alias] = series.resample(freq).ffill()
        print(f"✅ Loaded {alias}")
    except Exception as e:
        print(f"❌ Failed: {alias} - {e}")

for code, alias in fallback_fred_layman_dict.items():
    try:
        series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
        df[alias] = series.resample(freq).ffill()
        print(f"✅ Loaded {alias}")
    except Exception as e:
        print(f"❌ Failed: {alias} - {e}")

# 🧼 Fill any remaining gaps
df = df.bfill().ffill()

# Save
df.index.name = "date"
df.to_csv("layman_fred_batch.csv")
print("✅ Saved: layman_fred_batch.csv")

✅ Loaded layman__oil_price
✅ Loaded layman__gas_price
✅ Loaded layman__mortgage_rate_30yr
✅ Loaded layman__median_home_price
❌ Failed: layman__cpi_grocery_prices - Bad Request.  The series does not exist.
✅ Loaded layman__total_retail_sales
✅ Loaded layman__cpi_electricity
✅ Loaded layman__cpi_used_cars
✅ Loaded layman__total_consumer_credit
✅ Loaded layman__avg_auto_loan_rate_48mo
✅ Loaded layman__total_student_loans
✅ Loaded layman__rental_vacancy_rate
❌ Failed: layman__cpi_fast_food - Bad Request.  The series does not exist.
✅ Loaded layman__cpi_event_admission
✅ Loaded layman__cpi_airfare
❌ Failed: layman__cpi_childcare - Bad Request.  The series does not exist.
✅ Loaded layman__cpi_transit_fares
✅ Loaded layman__consumer_sentiment_index
✅ Loaded layman__personal_savings_rate
✅ Loaded layman__unemployment_rate
✅ Loaded layman__federal_funds_rate
✅ Loaded layman__youth_unemployment_rate
✅ Loaded layman__food_price_index
❌ Failed: layman__food_away_from_home_index - Bad Request.  The

In [11]:
df.columns

Index(['layman__oil_price', 'layman__gas_price', 'layman__mortgage_rate_30yr',
       'layman__median_home_price', 'layman__total_retail_sales',
       'layman__cpi_electricity', 'layman__cpi_used_cars',
       'layman__total_consumer_credit', 'layman__avg_auto_loan_rate_48mo',
       'layman__total_student_loans', 'layman__rental_vacancy_rate',
       'layman__cpi_event_admission', 'layman__cpi_airfare',
       'layman__cpi_transit_fares', 'layman__consumer_sentiment_index',
       'layman__personal_savings_rate', 'layman__unemployment_rate',
       'layman__federal_funds_rate', 'layman__youth_unemployment_rate',
       'layman__food_price_index', 'layman__general_cpi_index',
       'layman__medical_services_index'],
      dtype='object')

In [13]:
len(df.columns)

22